In [2]:
import pandas as pd
import sqlite3
import os

In [3]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/datasets-csv/restaurants.sql
/kaggle/input/datasets-csv/orders.csv
/kaggle/input/datasets-csv/users.json


Load orders.csv

In [4]:
orders = pd.read_csv("/kaggle/input/datasets-csv/orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


Load users.json

In [5]:
users = pd.read_json("/kaggle/input/datasets-csv/users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


Load restaurants.sql

In [7]:
conn = sqlite3.connect("restaurants.db")


In [8]:
with open("/kaggle/input/datasets-csv/restaurants.sql", "r") as file:
    sql_script = file.read()

conn.executescript(sql_script)


In [9]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


Merge Orders + Users

In [10]:
merged_data = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)


Merge With Restaurants

In [12]:
final_data = pd.merge(
    merged_data,
    restaurants,
    on="restaurant_id",
    how="left"
)


In [14]:
final_data.to_csv("final_food_delivery_dataset.csv", index=False)


In [15]:
data = pd.read_csv("final_food_delivery_dataset.csv")
data.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


MULTIPLE CHOICE QUESTIONS CODE

In [18]:
gold_orders = data[data["membership"] == "Gold"]


In [19]:
gold_orders.groupby("city")["total_amount"].sum().sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [20]:
data.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [21]:
bins = [3.0,3.5,4.0,4.5,5.0]

labels = ["3.0-3.5","3.6-4.0","4.1-4.5","4.6-5.0"]

data["rating_group"] = pd.cut(data["rating"], bins=bins, labels=labels)

data.groupby("rating_group")["total_amount"].sum()


/tmp/ipykernel_55/2893465760.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data.groupby("rating_group")["total_amount"].sum()


rating_group
3.0-3.5    1881754.57
3.6-4.0    1717494.41
4.1-4.5    1960326.26
4.6-5.0    2197030.75
Name: total_amount, dtype: float64

In [22]:
data.groupby("cuisine").agg({
    "restaurant_id":"nunique",
    "total_amount":"sum"
})


,restaurant_id,total_amount
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [23]:
total_orders = len(data)

gold_percent = (len(gold_orders) / total_orders) * 100

round(gold_percent)


50

In [25]:
data.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating', 'rating_group'],
      dtype='object')

In [27]:
restaurant_stats = data.groupby("name").agg({
    "total_amount": "mean",
    "order_id": "count"
})

restaurant_stats[restaurant_stats["order_id"] < 20].sort_values("total_amount", ascending=False)


,total_amount,order_id
name,,
User_2429,1497.42,1
User_889,1492.63,1
User_1843,1484.24,1
User_1882,1481.84,1
User_925,1476.18,1
...,...,...
User_2368,119.08,1
User_2583,117.62,1
User_1188,109.58,1


In [28]:
data.groupby(["membership","cuisine"])["total_amount"].sum().sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [29]:
data["order_date"] = pd.to_datetime(data["order_date"])

data["quarter"] = data["order_date"].dt.to_period("Q")

data.groupby("quarter")["total_amount"].sum()


/tmp/ipykernel_55/464516812.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data["order_date"] = pd.to_datetime(data["order_date"])


quarter
2023Q1    1993425.14
2023Q2    1945348.72
2023Q3    2037385.10
2023Q4    2018263.66
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

NUMERICAL QUESTIONS

In [30]:
gold_orders = data[data["membership"] == "Gold"]
len(gold_orders)


4987

In [31]:
hyd_revenue = data[data["city"] == "Hyderabad"]["total_amount"].sum()
round(hyd_revenue)


1889367

In [32]:
data["user_id"].nunique()


2883

In [33]:
gold_orders["total_amount"].mean().round(2)


np.float64(797.15)

In [34]:
len(data[data["rating"] >= 4.5])


3374

In [35]:
gold_city_revenue = gold_orders.groupby("city")["total_amount"].sum()

top_city = gold_city_revenue.idxmax()

len(gold_orders[gold_orders["city"] == top_city])


1337

FILL IN THE BLANKS

In [36]:
print("Join column:", "user_id")


Join column: user_id


In [37]:
print("File format:", "SQL")


File format: SQL


In [38]:
print("Function used:", "merge()")


Function used: merge()


In [39]:
restaurants.columns


Index(['restaurant_id', 'restaurant_name', 'cuisine', 'rating'], dtype='object')

In [40]:
print("Join key:", "restaurant_id")


Join key: restaurant_id


In [41]:
users.columns


Index(['user_id', 'name', 'city', 'membership'], dtype='object')

In [42]:
len(data)


10000

In [43]:
data.isnull().sum()


order_id               0
user_id                0
restaurant_id          0
order_date             0
total_amount           0
restaurant_name_x      0
name                   0
city                   0
membership             0
restaurant_name_y      0
cuisine                0
rating                 0
rating_group         331
quarter                0
dtype: int64